## Validation and Overfitting

- Shakedown する原因は大きく2つある
    1. Trust CV できなかった（検証データを無視し、パブリックに寄った誤った Submit を選択した）
    2. パブリックとプライベートに一貫性がなかったか、どちらかのデータセットが小さいサイズのものだった

- 上記の理由から、プライベートに最も適した Submit を選択することがコンペでは求められる。

### 概要

1. Validation と Overfit の概念を理解する
2. 安定したバリデーションを構築するためになされるべき分割の数を特定する
3. 学習データとテストデータの分割を繰り返す最も頻繁に用いられる方法をブレークダウンする
4. バリデーションに付き物の問題を議論する


**バリデーションの例**

- 将来モデルがどれほどの精度を持ちうるかを事前に予測される必要があり、それを確かめることを Validation と呼ぶ
- 現在手に入ってる（過去の）正解ラベル付きのデータを学習データと検証データに分割するのが一般的である
    - 将来のデータはテストデータで、これがどれくらいの精度を持ちうるかを知るために Validation を行う
- コンペではテストデータはさらにパブリックとプライベートに分割される

**Underfitting と Overfitting**

- Underfitting: モデルがシンプル過ぎて学習データへの精度すら不十分である（パラメータ不足?）
- Overfitting: モデルが複雑過ぎて学習データに適合しすぎてノイズに影響を受けやすい（パラメータ過多?）

- 一般的な過学習は、コンペへの過学習とは異なる
    - 一般的な過学習：学習データの精度は良いが、テストデータへの精度は良くない
    - コンペにおける過学習：テストデータの内、パブリックデータセットに寄りすぎることも過学習である
    
**まとめ**

1. バリデーションはモデルの品質を評価するのに役立つ
2. バリデーションによって未知のデータに対して最善の効果を発揮するモデルを選択することができる
3. Underfitting はデータのパターンを十分に捉えられていないことを指す
4. 一般的に Overfitting は、ノイズを捉え、テストデータに一般化されないパターンを捉えていることを指す
5. コンペにおいて Overfitting はテストデータに対して品質の低いモデルであることを指す

## Validation Strategies

### 基本のバリデーション分割手法

いくつかの異なるバリデーションを構築する手法をまとめる

**1. Holdout**

`sklearn.model_selection.ShuffleSplit`

- ngroups = 2
- データを学習データと検証データに分割する（Fold 1）
    - いずれのサンプルも学習と検証の両方に同時に使われてはならない。
        - バリデーションスコアが不当に高くなり、バリデーションスコアが信頼できなくなる。
        - オーバーサンプリングなど重複や繰り返しの方法を使う際には特に注意が必要である。
- ホールドアウト法は以下の場合に適している
    - データが十分にある場合
    - 計算効率の観点から K-Fold をした時に各Foldでほとんど同じスコアを出す場合（データが均質のため）（学習回数がK回ではなく1回のため）

**2. K-fold**

`sklearn.model_selection.Kfold`

- ngroups = k
- 学習と検証データに分割するのをK回繰り返す（検証データに割り当てられるのは一度のみ）
    - Holdout をK回繰り返すイメージ
    - 重要なのは、全てのサンプルが一度のみバリデーションに割り当てられるという点
        - 最終的に平均を取るので、十分にデータがなくモデルの精度や最適なパラメータが各Foldで不安定なときには KFold を行うのは良い選択である

**3. Leave-one-out**

`sklearn.model_selection.LeaveOneOut`

- ngroups = k
- Kfold の特殊なパターンで、Kとサンプル数が一致する場合
    - 学習データはK-1個、検証データは1個になる
    - データがとても少量しかないときに有用である
    
### Stratification

- 分類問題でランダムにシャッフルしても、不均衡データの場合にはうまく機能しないことがある
    - Fold ごとに割り当てられる正解ラベルの個数が不均衡になり、学習がうまくいかないため
- そのため異なる Fold に対して学習データ全体と同じ正解ラベルの分布を保存する Stratification が以下の場合に有用である。
    - 1. 少量のデータセット
    - 2. 不均衡なデータセット
    - 3. 多クラス分類
    
### Validation Scheme

**0. 基本ルール**

- モデルの精度を測るのに学習データを使ってはならない。全てのデータを学習データと検証データに分割することが必要。

**1. Holdout**

- データをAとBに分ける
- データAに対してモデルを fit させ、データBを予測する
- モデルの精度を計測するのにデータBの予測値を用いて、データBの予測値の精度を最大化するようなハイパーパラメータを見つける

**2. K-fold**

- 学習データ全体を K個の部分（Fold）に分割する
- ある Fold 以外のデータで学習し、その Fold に対して予測を行うことを、各 Fold で（K回）繰り返す
- 各 Fold の予測値を用いてモデルの精度を計算し、各 Fold に対して、精度を最大化するようなハイパーパラメータを見つける。Loss の平均と分散も記録しておくと、どれほどモデルが改善したかを理解するのにたいへん役立つ

**3. Leave-one-out**

- サンプル全体に対して、あるサンプル以外の全てのデータを用いてモデルを学習し、その一つのサンプルに対して予測を行うことを繰り返す。N回モデルを学習することになる。（Nはデータのサンプル数）
- 学習データの全てのサンプルへの予測値を得られるので、Loss を計算できる


## Data splitting strategies

- Validation の構築の仕方に関するより具体的な方法を見ていく
- **原則: Validation は、Train/Test の分割の関係性を再現するように構築しなければならない**

### 時系列データに対する Validation

- 時系列モデルではランダムな分割ではなく、時系列に沿った分割で検証データを作らなければならない
    - 重要な特徴量
        - 前後の目的変数の値
        - 時系列的なトレンド
    - 重要なモデル
        - 時系列で切ってモデルを構築しないと、CVスコアがテストスコアより不当に高くなる
            - 例えば単変量の単調な時系列データの場合、ランダムに分割すると学習データの目的変数の平均は検証データとの差とテストデータとの差で大きく異なる一方、時系列で分割すると両方とも差が大きくなるため

- 時系列で量的変数を目的変数とした Kaggle のコンペから得られた重要な示唆（ROSSMAN, BIMBO, etc.）
    - 異なる分割方法で以下の結果が大きく異なる
        - 1. 特徴量の生成
        - 2. モデルの特徴量への依存の仕方
        - 3. ある種のターゲットリーク
    - このため、どのように train/test が分割されているかを特定し、それを再現する必要がある
    
- データを Train と Validation に分割する方法
    - 1. ランダム
        - 各行は互いに独立であることを意味する
        - 逆に考えると、依存関係にある事象を特徴量に落とし込むと有効な特徴量となりうる
    - 2. 時系列
        - Train/Test が日時をもとに分割されている場合は特別なアプローチが必要となる
        - 特定の期間で区切った目的変数を利用した特徴量生成が有効な特徴量となりうる
            - 移動窓（Moving Window）バリデーションというアプローチが用いられる
    - 3. ID
        - IDごとに集約した値は、テストデータで新たに現れるIDに対しては無効である
        - IDベースのアプローチが必要だった過去のコンペでは、IDが隠されていた
            - IDが隠されていてもこれを無視してランダム分割をすると Overfit してしまうので、自らIDを抽出し補完する必要があった。
            - 例えばクラスタリングによって似たようなIDであると予測する手法が取られた
    - 4. 複合
        - 例えばショップごとの将来の売上を予測するタスクの場合、時系列での分割とIDでの分割が必要
        - あるいは異なる地域でユーザーが将来どのような音楽を聞くかを予測するタスクの場合も、同様

- **原則: Validation は、Train/Test の分割の関係性を再現するように構築しなければならない**
    - Depot の検索クエリと検索結果の関連性を予測するタスクのコンペでは、Train と Test で異なる検索クエリが与えられていたので、ランダムな分割も検索クエリによる分割も無効だった（前者は Overfitting に、後者は Underfitting）
    - テストデータに含まれる学習データにはない新しい検索クエリの比率を、Train/Validation の間で再現するようなデータ分割の仕方が望ましい
    
- まとめ
    - 多くの場合、データは以下の方法で分割できる
        - ランダム
        - 時系列
        - ID
    - 特徴量生成のロジックは、データ分割の戦略による
    - バリデーションセットはコンペにおける train/test の分割を再現するように構築する

## Problems occurring during validation

バリデーションの戦略とデータ分割の仕方を学んできて、  
これらを守ればCVとプライベートLBの結果が連動し、CVを改善すればPrivate LBが改善すると期待する。  
しかし、いくつかの場合ではそうならないことがある。  

- 1. Validation Stage
ローカルCVが不安定な問題。通常、幅広いデータが幅広いデータが各Foldごとに異なる最適パラメータを取ってしまうことに起因する。より綿密なバリデーション戦略が必要となる。

- 2. Submission Stage
手元CVとPublic LBが不一致になる問題。バリデーションにおいて、Train/Test の分割を正しく再現できていないことに起因する。これは難しい問題で、この章で詳しく見ていく。(どのような場合にShake（Public と Private で順位が大きく変動すること）が起きうるかについても見ていく)


### 1. Validation Stage

このステージでの問題は、各学習FoldごとにCVスコアと最適パラメータが大きく異なることである  
- 例えば2月の売上を予測するのに、1月の売上を利用する場合を考えると、1月は2月に比べて休日が多いにもかかわらず1月の休日を特徴量に加えずモデルを構築すると学習データへの過学習を起こし、CVスコアは向上しない。
- ここから考えられるCVスコアの不安定性の要因は以下のものが想定される
    - A. データが小さすぎる（モデルが限られたパターンのみしか一般化できなくなるため）
    - B. データが多様すぎたり、一貫してなかったりする（似たようなデータが大きく異なるTargetを取るとモデルが混乱するため）  
    
この問題に直面したとき、バリデーションを拡張するべきである（Extensive Validation）  
- A. 異なる random_seed での KFold-Splits を複数回行い、そのスコアを平均する（いわゆる Seed Averaging?）
    - より安定したモデルの品質を手に入れることができる
- B. ある Split でチューニングしたモデルのパラメータを、異なる Split でのモデルのパラメータとして用いる。
    - Overfit を起こしそうな時に、最適なパラメータを選択することができる
        
コンペにおける Extensive Validation の例
- Liberty Mutual Group: Property Inspection Prediction
- Santander Customer Satisfation
- これらのコンペでは参加者のスコアがとても拮抗にしていたので、Overfit を避けながらデータから以下に重要な情報を抽出しきれるかが重要になった。そこで綿密なバリデーション戦略が求められた。


### 2. Submission Stage

以下のような問題に直面することがある
- LBスコアがCVスコアと継続的に大きくなる（小さくなる）
- LBスコアがCVスコアの改善・後退と全く関連しない

ここで信頼たる Validation を構築するための原則「Train/Test の分割を再現する」を思い出す
- ただ実際に、正確に Train/Test の分割を再現することは非常に困難である。
- そのためコンペに参加したら早めにベースラインとなるモデルを提出することを強く推奨する。

Submission Stage でCV戦略がうまく機能しない場合の原因の探索方法は以下のようなものが考えられる
- 0. CVスコアの計算が間違っていないかを確認する
    - Validation Stage で異なる Random Seed で得られた異なる KFold の結果を LB にサブミットしてみる。
    - Validation で異なるCVスコアを得ていれば、LB でも同様に異なるスコアを得ることは想像できる。このときに各々のスコアの平均と標準偏差を計算しておき、それらに大きく乖離がないかを確認するのが良い。
    - ここで大きく乖離がある場合は、確実に何かが間違っている

上記とは別の問題の原因の可能性
- 1. パブリックLBでのデータがとても小さい
    - この場合は、Truct Your CV する。
- 2. Train と Test のデータが異なる分布である
    - 変数の分布が異なる場合、どうしてもTrainの学習データの分布に過学習して、Testに対するスコアがヒドイものになる
    - この場合、学習の前処理を通して適合させる方法とパブリックLBを通して適合させる方法(LB Probing)がある
        - Train/Test で常に最適な予測をするものを見つけ出し、その差分の分だけ予測をシフトさせる方法
        - 2つの constant なサブミットを行い、その結果を記録することで、Test の平均値を知ることができる(LB Probing)
        - ただし、基本は Validation スキームに従うこと。今回のケースのように LB Probing なしでは Validation スキームを形成できない場合も時にはある。
        - Quora Question Pairs コンペはそのようなコンペの一つであった（このようなコンペは珍しい）
        
よくあるケースは極端な場合ではなく、ターゲットの値があべこべに不均衡である場合
- この場合も、原則どおり、Validation をテストデータを再現するように分割する必要がある
- そうすることで Test データにとっても最適なパラメータを選択することができる
- 似たようなケースのコンペとして、Data Science Game 2017 Qualification phase: Music recommendation や CTR Prediction task from RDA などがある。

CTR予測のコンペの例を取ってみる
- 問題の前提として、学習データでは特定の広告のみしか学習できない一方、テストデータはあらゆる広告の可能性がある。
- そのためここでも原則どおり、Validation は Test を再現するように構築する必要がある。つまり、Validation には学習データに現れないようなデータを含めるように構築しなければならない。


### LB Shuffle (Shake)

- LB Shuffle(Shake) とは多くの参加者の LB のパブリックとプライベートが大きく変化することを指す。
- Shake の起こる要因は主に3つ
    - ランダム性
        - 多くの参加者がかなり似たようなスコアの場合に起こりうる。単にランダム性によるスコアの差がついているだけである。
        - Two Sigma Financial Model (Kaggle の中でも最大の Shake が起こったコンペの一つ)
        - Liberty Mutual Group Property Inspection Prediction competition
    - データが少なすぎること
        - データが少ないこと、特にプライベートのテストデータが少ない場合に起こりやすい。
        - TFI　Restaurant Revenue Prediction Competition
            - 学習データ200、テストデータが400のコンペだった
    - Public と Private の分布が異なること
        - 時系列データのコンペでよくある
        - Rossman Store Competition
            - パブリックとプライベートが時系列で分割されていたため、パブリックに適合させると Overfit で Shake Down を食らった。ここでも **Trust Your Validation(and Everything will be fine)** である。


### まとめ

- Validation 段階でスコアの大きな乖離がある場合、Extensive Validation（拡張バリデーション）を行うべき
    - 異なる Random Seed で複数の KFold Splits によるスコアの平均を用いること（Seed-Averaging）
    - ある Split でモデルのパラメータをチューニングして、別の Split でそのモデルパラメータを用いてスコアを評価する
    
- パブリック LB スコアがローカルの Validaiton スコアと一致しない場合以下のことを確認すべきである
    - パブリック LB のデータが少なすぎないか
    - オーバーフィットしていないか
    - 正しい Validation Split の戦略の取っているか
    - Train/Test が異なる分布をしていないか
    
- 以下のような場合には LB Shuffle が予想される
    - ランダム性がモデルの大きな要因になる場合
    - データが少量の場合
    - Public / Private で異なる分布を保つ場合

## Additional materials and links

- [Validation in sklearn](http://scikit-learn.org/stable/modules/cross_validation.html)
- [Advices on validation in a competition](http://www.chioka.in/how-to-select-your-final-models-in-a-kaggle-competitio/)